In [4]:
import numpy as np
import pandas as pd

# Helper Functions

In [5]:
def output_data(file, algorithm):
    data = pd.read_csv(file, sep=" ", header=None)
    data.columns = ["k", "s", "cost", "file", "algorithm"]
    data['algorithm'] = algorithm
    return data

# Comparing Ant Colony and 2-opt

In [6]:
tsp_100 = output_data('best_tsp_100.txt', 'tsp')
print(tsp_100.shape)
tsp_100.head()

(316, 5)


,k,s,cost,file,algorithm
0,1,1,6.492717e+07,inputs/131_100.in,tsp
1,7,7,7.400000e+01,inputs/70_100.in,tsp
2,1,1,9.200000e+01,inputs/286_100.in,tsp
3,4,4,1.481217e+09,inputs/152_100.in,tsp
4,12,11,1.195667e+03,inputs/13_100.in,tsp


# Conclusion
2-opt does better than Ant Colony on average.

## Merge *_50.in Inputs

In [9]:
import shutil
import os

def copy_outputs(df, output_dir_old, output_dir_new):
    if not os.path.exists(output_dir_new):
        os.makedirs(output_dir_new)
    for file in df['file'].tolist():
        file_code = file.split("/")[1].split('.in')[0]
        shutil.copy2(str(output_dir_old) + str(file_code) + '.out', str(output_dir_new))
        
def merge_best_outputs(ant_file, tsp_file, ant_out_dir, tsp_out_dir, final_out_dir):
    if ant_file and tsp_file:
        ant_data = output_data(ant_file, 'ant')
        tsp_data = output_data(tsp_file, 'tsp')
        merged_tsp_ant = pd.merge(ant_data, tsp_data, left_on='file', right_on='file', suffixes=('_ant', '_tsp'))
        ant_better = merged_tsp_ant[merged_tsp_ant['cost_ant'] < merged_tsp_ant['cost_tsp']]
        tsp_better = merged_tsp_ant[merged_tsp_ant['cost_ant'] >= merged_tsp_ant['cost_tsp']]
        copy_outputs(ant_better, ant_out_dir, final_out_dir)
        copy_outputs(tsp_better, tsp_out_dir, final_out_dir)
    elif tsp_file and not ant_file:
        tsp_data = output_data(tsp_file, 'tsp')
        copy_outputs(tsp_data, tsp_out_dir, final_out_dir)

In [10]:
merge_best_outputs('best_ant_50.txt', 'best_tsp_50.txt', 'outputs_ant_50/', 'outputs_tsp_50/', 'outputs/')